<img src="https://s3-sa-east-1.amazonaws.com/preditiva.ai/diversos/preditiva_assinatura.jpg">

# Computer Vision

Referências:
* https://www.kaggle.com/mloey1/ahdd1?select=csvTestLabel+10k+x+1.csv

## Hands On: Dígitos Escritos a Mão em Árabe

### Importação das bibliotecas

In [ ]:
# Pacotes de preparação dos dados
import pandas as pd
import numpy as np

# Pacotes gráficos
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.cm as cm
import seaborn as sns

# Pacotes de Modelagem
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.utils import np_utils

# Funções adicionais
import preditiva

### Importação da base de dados

In [ ]:
# Load the data
df_x_train = pd.read_csv("csvTrainImages 60k x 784.csv", header=None)
df_y_train = pd.read_csv("csvTrainLabel 60k x 1.csv", header=None)
df_x_test = pd.read_csv("csvTestImages 10k x 784.csv", header=None)
df_y_test = pd.read_csv("csvTestLabel 10k x 1.csv", header=None)

print(df_x_train.shape)
print(df_y_train.shape)
print(df_x_test.shape)
print(df_y_test.shape)

In [ ]:
df_x_train

### Separação dos dados da imagem dos Rótulos

In [ ]:
x_train = df_x_train.values.astype('float32')
x_test  = df_x_test.values.astype('float32')

y_train = df_y_train.values.astype('int32')
y_test  = df_y_test.values.astype('int32')

### Análise Exploratória de Dados

In [ ]:
# Conjunto de Treino
preditiva.cria_grafico_var_qualitativa(preditiva.tabela_bivariada(pd.DataFrame(y_train, columns=['y_train']), 'y_train'))

In [ ]:
# Conjunto de Teste
preditiva.cria_grafico_var_qualitativa(preditiva.tabela_bivariada(pd.DataFrame(y_test, columns=['y_test']), 'y_test'))

### Visualização das imagens

In [ ]:
x_train.shape

In [ ]:
x_train = x_train.reshape(x_train.shape[0], 28, 28)

plt.figure(figsize=(5,7))

for i in range(9):
    plt.subplot(330 + (i+1))
    plt.imshow(x_train[i], cmap=plt.get_cmap('gray'))
    plt.title(y_train[i])

In [ ]:
def visualize_input(img, ax):
    ax.imshow(img, cmap='gray')
    width, height = img.shape
    thresh = img.max()/2.5
    for x in range(width):
        for y in range(height):
            ax.annotate(str(round(img[x][y],2)), xy=(y,x),
                        horizontalalignment='center',
                        verticalalignment='center',
                        color='white' if img[x][y]<thresh else 'black')

obs = 42
fig = plt.figure(figsize = (18,18)) 
ax = fig.add_subplot(111)
visualize_input(x_train[obs], ax)

### Pré-processamento

In [ ]:
x_train = x_train.astype('float32')/255.0
x_test  = x_test.astype('float32')/255.0

In [ ]:
x_train = x_train.reshape(-1,28,28,1)
x_test = x_test.reshape(-1,28,28,1)

In [ ]:
# Criação das variáveis dummy
y_train = np_utils.to_categorical(y_train, 10)
y_test = np_utils.to_categorical(y_test, 10)

## Definição da CNN

In [ ]:
model = keras.Sequential()

# 1a Camada: Convolução
model.add(layers.Conv2D(10,
                        kernel_size=5, 
                        activation='relu', 
                        input_shape=(28,28,1)))
model.add(layers.Dropout(0.3))

# 2a Camada: MaxPolling
model.add(layers.MaxPooling2D())

# 3a Camada: FullyConnected / Dense / MLP
model.add(layers.Flatten())
model.add(layers.Dense(10, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer=keras.optimizers.Adam(), 
              metrics=[keras.metrics.CategoricalAccuracy()])

model.summary()
print("Input shape ",model.input_shape)
print("Output shape ",model.output_shape)

## Treinamento da CNN

In [ ]:
cnn_es = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    min_delta=0.01,
    patience = 10,
    verbose=2,
    mode='min',
    restore_best_weights=True)

historico = model.fit(x=x_train, 
                      y=y_train,
                      epochs=50,
                      batch_size=512,
                      validation_split=0.1,
                      callbacks=cnn_es,
                      verbose=1)

## Avaliação do Desempenho

In [ ]:
score, metric_train = model.evaluate(x_train, y_train, verbose=0)
score, metric_test = model.evaluate(x_test, y_test, verbose=0)

print('Desempenho no conjunto de Treino: {:.2%}'.format(metric_train))
print('Desempenho no conjunto de Teste: {:.2%}'.format(metric_test))

In [ ]:
# Gráfico com a evolução do treinamento
fig, ax = plt.subplots(figsize=(8, 8))
plt.style.use('ggplot')
plt.plot(historico.history['categorical_accuracy'])
plt.plot(historico.history['val_categorical_accuracy'])
plt.title('Desempenho do Modelo')
plt.ylabel('Acurácia')
plt.xlabel('Épocas')
plt.legend(['Treino', 'Validação'], loc='upper left')
plt.show()